### Imports

In [1]:
import json
import ee
import geemap
import folium
import os
import pandas as pd
import geopandas as gpd
import time
import math
import numpy as np
from datetime import datetime
from sidecar import Sidecar

from modules.json_to_ee import json_to_feature_with_id
from modules.agstack_setup import start_agstack_session
import modules.agstack_to_gee as agstack_to_gee
import modules.area_stats as area_stats
import modules.tidy_tables as tidy_tables
from parameters import * # for run-specific parameters edit "parameters/config_runtime"

print ("imports complete")

imports complete


In [2]:
ee.Initialize()

### 1. Grab datasets


In [4]:
if use_existing_image_collection:
    images_IC = ee.ImageCollection("users/andyarnell10/fdap_dpi/imageCol_trial_2")
    print ("using existing image collection asset")
else:
    from dataset_properties.set_image_properties import images_IC
    print ("compiling image collection on the fly")

## get lists from lookup for different streams of processing
from  dataset_properties.make_processing_lists_from_lookup import local_buffer_stats_list,\
                                                                  flag_list,\
                                                                  country_allocation_stats_only_list, \
                                                                  normal_poly_stats_list, \
                                                                  decimal_place_column_list


Starting image prep. This can take a minute
scale (m):  10
number of images:  17
Total execution time: 2.0096445083618164 seconds
compiling image collection on the fly


### 2. Fetch some fields (public)

#### Transform geometries into a feature collection

In [5]:
CIV_ids = ['0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f665ea83ac9aad3512918',
           'b84f55de2b7f3c77d1cbeb8b026a1b29be42d8b08d92058c9143e0556456820f',
           'b7c15efb6e3c63fcfe649a2d994973a6f5caa844f720f0edb7cf24f6a6c3c1b3',
            'fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b84f5b9445855eef02f']

GHA_ids = ['88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907', 
        'ef2f7c46fbe4fc892fdb81f9a31c9c507b9f1e4548504247dcbbab28cf8e436c',
        '97408ef7bdac487e4a42e4abf20492b786310889fd4b0478603e2d0004c40bfb']

IDN_ids = ['c288d6c94efa9011c0e3452af9f7fa0941661377030e10d29c68764617f9816d', 
       '1a41a309ae2387f36a604c9a6c81887e64357a7f61d228758e23ef766286fcd7',
       '1a4472dc40700ef33f931863f58d444f243d64418616678fcf85c57e1f4bbf45',
       '8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53859993522a6f05727a']

all_geo_ids= CIV_ids+GHA_ids+IDN_ids

if debug: print (all_geo_ids)

['0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f665ea83ac9aad3512918', 'b84f55de2b7f3c77d1cbeb8b026a1b29be42d8b08d92058c9143e0556456820f', 'b7c15efb6e3c63fcfe649a2d994973a6f5caa844f720f0edb7cf24f6a6c3c1b3', 'fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b84f5b9445855eef02f', '88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907', 'ef2f7c46fbe4fc892fdb81f9a31c9c507b9f1e4548504247dcbbab28cf8e436c', '97408ef7bdac487e4a42e4abf20492b786310889fd4b0478603e2d0004c40bfb', 'c288d6c94efa9011c0e3452af9f7fa0941661377030e10d29c68764617f9816d', '1a41a309ae2387f36a604c9a6c81887e64357a7f61d228758e23ef766286fcd7', '1a4472dc40700ef33f931863f58d444f243d64418616678fcf85c57e1f4bbf45', '8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53859993522a6f05727a']


#### Start session

In [6]:
session = start_agstack_session(email,password,user_registry_base,debug)

Cookies <RequestsCookieJar[]>
status code: 500


#### Fetch and prepare features

In [7]:
#fetch and convert into feature collection
roi = agstack_to_gee.geo_id_or_ids_to_feature_collection(
    all_geo_ids, geo_id_column, session, asset_registry_base, debug)

roi = area_stats.add_area_hectares_property_to_feature_collection(roi,geometry_area_column)

#select only columns of interest
roi  = roi.select([geometry_area_column,geo_id_column]) 

#Create additional buffer zones for deforestation risk 
roi_alerts_buffer = roi.map(lambda feature: 
        feature.buffer(local_alerts_buffer_radius,max_error_alert_buff)) 


input: list
Count of geo ids in list:  11
Count of features in FeatureCollection:  11


### 3. Compute statistics

Calculating zonal statistics for continuous data (e.g tree cover) within polygon(s)

##### i) Mapping over image collection with reduce regions (creates long format temporary table)

In [8]:
# get the start time
st = time.time()
if debug: print ("processing stats...")

## reducer choice for zonal statistics
reducer_choice = ee.Reducer.sum().combine(  #main stats based on area of pixel
  reducer2=ee.Reducer.count(),sharedInputs=True).combine(
    reducer2=ee.Reducer.mode(), sharedInputs=True) ## mode for country allocation (majority pixel count) 

## get stats for roi (not including deforestation alerts)
fc_stats_poly = area_stats.reduceStatsIC(roi,
                                  images_IC.filter(ee.Filter.inList("system:index",normal_poly_stats_list)),
                                                                  reducer_choice)# all but alerts
## get stats for buffer (alerts only)
fc_stats_buffer = area_stats.reduceStatsIC(roi_alerts_buffer,
                                                    images_IC.filter(ee.Filter.inList(
                                                    "system:index",local_buffer_stats_list)),
                                                    reducer_choice) #alerts only

#combine stats from roi and buffer into one feature collection
fc_stats_combined = fc_stats_poly.merge(fc_stats_buffer) 

# convert to Pandas Dataframe
df_combined = geemap.ee_to_pandas(fc_stats_combined) # limit of 5000 (unlikely to need more but i have code for it if needed)

# export dataframe to csv
df_combined.to_csv(path_or_buf=out_file_long,header=True,index=False)

# get the execution time
elapsed_time = time.time() - st

if debug: print ('Total execution time:', elapsed_time, 'seconds')

processing stats...
Total execution time: 2.745312213897705 seconds


##### ii): Create lookup tables for country allocation
Approach is based on raster stats and listing the country for a specific geometry based on which has most overlap



Look up table linking country codes to country names (from GAUL feature collection) is stored here: scripts: create_country_lookup.py

Make on-the-fly look up table to link country name to geo id based on raster stats
- uses rasterised GAUL layer with admin codes as pixel values
- for each geo id finds most common value in that geometry (i.e. "mode" statistic)

In [9]:
fc_stats_country_codes = area_stats.reduceStatsIC(roi,
                                  images_IC.filter(ee.Filter.eq("dataset_id",16)),
                                  reducer_choice)# all but alerts

df_stats_country_codes = geemap.ee_to_pandas(fc_stats_country_codes) # limit of 5000 (unlikely to need more fpr demo but i have code for it if this happens)

lookup_geo_id_to_GAUL_codes = df_stats_country_codes[df_stats_country_codes["dataset_name"]=="GAUL_boundaries_adm0_code_reproj"]  #get mode stats for GAUL dataset

lookup_geo_id_to_GAUL_codes = lookup_geo_id_to_GAUL_codes[[geo_id_column, 'mode']] # choose only columns needed

lookup_geo_id_to_GAUL_codes = lookup_geo_id_to_GAUL_codes.rename(columns={"mode":"ADM0_CODE"}) # change names for a clean join 

lookup_geo_id_to_GAUL_country_names = lookup_geo_id_to_GAUL_codes.merge(lookup_country_codes_to_names,on="ADM0_CODE",how="inner").drop("ADM0_CODE",axis=1) # join geo id to the GAUL_lookup_table countaining "Country_names"
# lookup_geo_id_to_ISO3 = lookup_geo_id_to_GAUL_codes.merge(lookup_country_codes_to_ISO3,on="ADM0_CODE",how="inner").drop("ADM0_CODE",axis=1) # join geo id to the GAUL_lookup_table countaining "Country_names"


##### iii) Reformat results table
- long to wide
- convert to proportions 
- set presence only flags
- add in country names (using lookup tables) to the final results

In [10]:
#add proprtion column
df_combined["percentage"] = (df_combined["sum"]/df_combined[geometry_area_column])*100

#convert to wide format (one row per geo_id)
df_wide_format = df_combined.pivot_table(index=[geo_id_column,geometry_area_column],columns=['dataset_name'],values=['percentage'])

#tidy unwanted headers etc
tidy_tables.tidy_dataframe_after_pivot(df_wide_format) #runs in place so no need to assign

# convert positive results values to "True" for specific columns
for column in flag_list: df_wide_format[column]=np.where(df_wide_format[column]>0,"True","-")

# # tidy output - decimal places
for column in flag_list: df_wide_format[decimal_place_column_list]=df_wide_format[decimal_place_column_list].round(decimals=0, out=None).astype(int)

df_wide_format=df_wide_format.reset_index()

df_wide_format[geometry_area_column]=df_wide_format[geometry_area_column].round(decimals=1, out=None)

# #joins country name based on majority overlap with country 
# if debug: print (columns_list)
print(decimal_place_column_list)
print(flag_list)

df_wide_format = df_wide_format.merge(lookup_geo_id_to_GAUL_country_names,on=geo_id_column)

['GFC_Tree_Cover_2020', 'ESRI_Trees_2020', 'JAXA_Forest_non_forest_2020', 'GLAD_LULC_Stable_Tree_2020', 'TMF_undisturbed_forest_2020', 'Primary_Humid_Tropical_Forest_2020', 'TMF_disturbed_forest_2020', 'TMF_plantation', 'Oil_palm_Descals', 'FDaP_palm_plantations', 'Cocoa_plantations_Kalischek', 'JRC_GFC_2020']
['Local_RADD_alerts', 'Protected_area', 'Other_Effective_area_based_Conservation_Measure', 'Key_Biodiversity_Area']


##### iv) Further reformatting and exporting
- reorder columns
- remove underscores in column titles
- export to csv

In [11]:
# reorder columns using list 
ordered_dataset_df= lookup_gee_datasets.sort_values(by=['datasets_order'])

column_order_list = list(ordered_dataset_df["dataset_name"])
                         
column_order_list.insert(0,geo_id_column) # add in the "geo_id" column into to datasets list

column_order_list.insert(1,geometry_area_column)# add in to list the geometry area column

column_order_list.remove("GAUL_boundaries_adm0_code_reproj") # remove old column with "mode" values (not now relevant as have country names)

column_order_list.insert(2,"Country")# add in to list the new column with country names

df_wide_format= df_wide_format.reindex(columns=column_order_list) # reorder by list

df_wide_format["Country"]=np.where(df_wide_format["Country"]=="C�te d'Ivoire","Côte d'Ivoire",df_wide_format["Country"])# TEMP fix on characters (encoding issues)
df_wide_format["Country"]=np.where(df_wide_format["Country"]=="R�union","Réunion",df_wide_format["Country"])# TEMP fix on characters (encoding issues)

# remove underscores in columns
df_wide_format.columns = df_wide_format.columns.str.replace('_', ' ')

# #export wide format csv
df_wide_format.to_csv(path_or_buf=out_file_wide,header=True)

# if debug: print ("output csv: ", out_file_wide)

#checks
if debug: flag_list
# if debug: print (columns_list)

#df_wide_format

### 4. Convert to CEO input

In [12]:
df = pd.merge(geemap.ee_to_geopandas(roi), 
              df_wide_format, 
              left_on='Geo_id', 
              right_on='Geo id', 
              how='inner')

In [13]:
df["PLOTID"] = df.index +1
df.set_index("PLOTID", inplace = True)

In [14]:
ceo_cols = df_wide_format.columns

In [15]:
gdf = gpd.GeoDataFrame(df[ceo_cols],
                       geometry=df.geometry,
                       crs="EPSG:4326")

In [16]:
gdf.columns = ["geoid","pol_area","country","gfc_2020","esri_2020","jaxa_fnf",
               "glad_lulc","tmf_undist","glad_prim","tmf_dist","radd","tmf_plant","palm_desc","palm_fdap","cocoa_eth","wdpa","oecm","biodiv","geometry","jrc_2020"]

ValueError: Length mismatch: Expected axis has 20 elements, new values have 19 elements

In [ ]:
# using dictionary to convert specific columns
convert_dict = {"geoid": object,
                "gfc_2020": int,
                "esri_2020": int,
                "jaxa_fnf": int,
                "glad_lulc": int,
                "tmf_undist": int,
                "glad_prim": int,
                "tmf_dist": int,
                "radd": object,
                "tmf_plant": int,
                "palm_desc": int,
                "palm_fdap": int,
                "cocoa_eth": int,
                "wdpa":object,
                "oecm": object,
                "biodiv": object,
                "jrc_2020",int
                }
        
gdf = gdf.astype(convert_dict)
#print(gdf.dtypes)

In [ ]:
gdf.to_file("test_ceo_all.shp")

### 5. Create Map 

##### Loop through image collection and loads layers to be added to the map


In [17]:
images_IC = images_IC.sort("dataset_id").filter(ee.Filter.neq("system:index","GAUL_boundaries_adm0_code_reproj"))

In [18]:
Map = geemap.Map()
visParams =  {'min': 0,'max': 1,'palette':['White','Green']}

sc = Sidecar(title='Check Plots')
with sc:display(Map)

for i in range(images_IC.size().getInfo()):
    
    image_new = ee.Image(images_IC.toList(100,0).get(i))
    
    dataset_name = image_new.get("system:index").getInfo()
    
    if debug: print ("adding image",i,"-",dataset_name)
    Map.addLayer(image_new.gt(0).unmask(),visParams,dataset_name,0,1)
    
    
Map.addLayer(roi,{},'roi ',1,1)
# Map.addLayer(roi_alerts_buffer,{},'roi buffer zone')

if debug: print ("All layers added")    

adding image 0 - GFC_Tree_Cover_2020
adding image 1 - ESRI_Trees_2020
adding image 2 - JAXA_Forest_non_forest_2020
adding image 3 - GLAD_LULC_Stable_Tree_2020
adding image 4 - TMF_undisturbed_forest_2020
adding image 5 - Primary_Humid_Tropical_Forest_2020
adding image 6 - TMF_disturbed_forest_2020
adding image 7 - Local_RADD_alerts
adding image 8 - TMF_plantation
adding image 9 - Oil_palm_Descals
adding image 10 - FDaP_palm_plantations
adding image 11 - Cocoa_plantations_Kalischek
adding image 12 - Protected_area
adding image 13 - Other_Effective_area_based_Conservation_Measure
adding image 14 - Key_Biodiversity_Area
adding image 15 - JRC_GFC_2020
All layers added


### 6. Zoom on polygon

##### Show on map and zoom to a specific feature based on index number in feature collection
 Layers visibility off by default - toggle on in top right corner

In [19]:
# number/index from list of ROI features - the selected feature is shown on the map. e.g., choose 0 for first in the list 
plotid =2

#choose how close to zoom to chosen polygon (1-24, where 24 is fully zoomed in) 
zoom_level = 16 

single_feature_id = roi.aggregate_array(geo_id_column).get(plotid-1).getInfo()
if debug: print (geo_id_column,single_feature_id)
single_feature = ee.Feature(roi.filter(ee.Filter.eq(geo_id_column,single_feature_id)).first())

Map.centerObject(single_feature,zoom_level)
    
#Map

Geo_id b84f55de2b7f3c77d1cbeb8b026a1b29be42d8b08d92058c9143e0556456820f


### 7. Display table

In [ ]:
df[ceo_cols]

### 8. Generate CEO URL

https://app.collect.earth/collection?projectId=39849

### Logout (protected)

In [ ]:
# res = session.get(asset_registry_base + "/logout")
# if debug: print (res.json())
# res = session.get(user_registry_base + "/logout", cookies=session.cookies)
# session.headers.clear()

### Checking if Logged out correctly

In [ ]:
# # Confirming the logout from Asset Registry by requesting a Protected route
# req_body = {
#     "latitude": 31.47704430446457,
#     "longitude": 74.37510786779589
# }
# res = session.post(asset_registry_base + "/fetch-fields-for-a-point", json=req_body)
# if debug: print (res.json())

### Get all Domains (public)

In [ ]:
# # Fetching all the domains from the User Registry
# res = session.get(asset_registry_base + "/domains")
# if debug: print (res.json())